跟前一个版本的比较，这个版本的y是涨跌幅度

In [1]:
import sys
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import talib
import torch.utils.data as Data
# 这里要试试绘图
# 这里要试试绘图
from visdom import Visdom
from PIL import Image

# 获取数据

In [2]:
sys.path.append('../../')
import  DataSource
zz500 = DataSource.get_zz500_codes() # 我看中证500的数据
# 我这里查看第一个股票的吧
N = 60 # 多少日的数据
# 我这里仅仅是记录如下的列的涨跌幅度
rate_columns = ['open', 'high', 'low', 'close']
close_index = rate_columns.index('close')
input_size = len(rate_columns)
seq_len = N    # 多少个时间序列的股票
output_size = 1 # 我只是输出一个今天的收盘价相比较昨日的涨跌比率
num_layers = 10 # 多少个gru合并
hidden_size = 20 # 隐藏层的宽度
batch_size = 16 # 一个批次有多少个数据
lr = 0.001

In [3]:
def get_loader(code_name, N=60, rate_columns=['open', 'high', 'low', 'close'], batch_size=16, train_rate=0.9):
    dt = DataSource.get_data(code_name)
    dt_list = []
    for j in range(N):
        # N+1表示要多一天，比如前面30天，我要看看第31天的收盘价相对于第一天的收盘价是什么比例
        for i in rate_columns:
            dt_tmp = pd.DataFrame({f'{i}_rate_{j}':(dt[f'{i}'].shift(-j) - dt[f'preclose'])/dt[f'preclose']})
            dt_list.append(dt_tmp)
    dt2 = pd.concat(dt_list[:-(len(rate_columns))], axis=1)
    # x
    x_price = dt2.to_numpy()
    x_price = x_price[:-(N+1)]
    # 更改维度
    x_shape_old = x_price.shape
    x_shape_new_1 = x_shape_old[0]
    x_shape_new_3 = len(rate_columns)
    x_shape_new_2 = int(x_shape_old[1] / x_shape_new_3)
    x_price = x_price.reshape((x_shape_new_1, x_shape_new_2, x_shape_new_3))
    # y，是涨跌幅度
    dt['close2'] = dt['close'].shift(-N)
    dt['close3'] = dt['close'].shift(-(N+1))
    dt['close4'] = (dt['close3']-dt['close2'])/dt['close2']
    y_close = dt['close4'].to_numpy()[:-(N+1)]
    # 然后是批次的整数倍
    _x_y_len = int(x_price.shape[0]/batch_size) * batch_size
    #
    x_price = x_price[-_x_y_len:]
    y_close = y_close[-_x_y_len:]
    x_price = x_price.reshape(
        x_price.shape[0]//batch_size,
        batch_size,
        x_price.shape[1],
        x_price.shape[2])
    y_close = y_close.reshape(y_close.shape[0]//batch_size, batch_size, 1)
    # 做成加载器
    dataset = Data.TensorDataset(
        torch.FloatTensor(x_price),
        torch.FloatTensor(y_close))
    train_loader, test_loader = Data.random_split(
        dataset,
        lengths=[
            int(train_rate*len(dataset)),
            len(dataset)-int(train_rate*len(dataset))],
        generator=torch.Generator().manual_seed(0))
    return train_loader, test_loader

In [4]:
# 一个gru网络
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(GRUModel, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.gru(x)  # gru层
        out = self.fc(out[:, -1, :])  # 全连接层
        return out

In [5]:
# 训练
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = GRUModel(input_size, hidden_size, num_layers, output_size).to(device)
criterion = torch.nn.L1Loss()  # 绝对值误差函数
optimizer = optim.Adam(net.parameters(), lr=lr)

In [6]:
# 获取绘图对象，相当于plt
vis = Visdom()

Setting up a new session...


In [7]:
def do_train(epoch_count, loader, train_line_name='train_line'):
    vis.line([0.], [0], win=train_line_name, opts=dict(title=train_line_name))
    _step = 1
    for epoch in range(epoch_count):  # 迭代
        arr_loss = []  # 记录损失
        arr_labels = []  # 实际值的平均
        for i, data in enumerate(loader, 0):
            # 获取输入
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            # 正向传播，反向传播，优化
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # 这里更新到显示
            vis.line([loss.cpu().detach().numpy()], [_step], win=train_line_name, update='append')
            _step = _step + 1
            arr_loss.append(loss.item())
            labels2 = torch.abs(labels)
            arr_labels.append((sum(labels2)/len(labels2)).item())
        loss_arg = sum(arr_loss)/len(arr_loss)
        labels_arg = sum(arr_labels)/len(arr_labels)
        print(f'{epoch+1}, avg labels: {labels_arg:.5f} avg loss:{loss_arg:.5f}, max loss: {max(arr_loss):.5f}, min loss : {min(arr_loss):.5f}')


def do_test(loader, test_line_name='test_line'):
    _step = 1
    vis.line([0.], [0], win=test_line_name, opts=dict(title=test_line_name))
    with torch.no_grad():
        for i, data in enumerate(loader, 0):
                # 获取输入
                inputs, labels = data
                inputs = inputs.to(device)
                labels = labels.to(device)
                # 正向传播，反向传播，优化
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                # 这里更新到显示
                vis.line([loss.cpu().detach().numpy()], [_step], win=test_line_name, update='append')
                _step = _step + 1
    print(f'实际值:{labels}')
    print(f'预测值:{outputs}')
    print(f'test :{loss.item()}')

In [8]:
# 创建窗口并初始化

train_loader, test_loader = get_loader(
    zz500[0],
    batch_size=batch_size,
    train_rate=0.9)
# 我这里用测试集来测试
do_train(10, train_loader)
# 接下来做测试
do_test(test_loader)
print('finish')


1, avg labels: 0.01464 avg loss:0.01683, max loss: 0.09261, min loss : 0.00584
2, avg labels: 0.01464 avg loss:0.01524, max loss: 0.07560, min loss : 0.00600
3, avg labels: 0.01464 avg loss:0.01511, max loss: 0.07519, min loss : 0.00596
4, avg labels: 0.01464 avg loss:0.01503, max loss: 0.07505, min loss : 0.00580
5, avg labels: 0.01464 avg loss:0.01503, max loss: 0.07578, min loss : 0.00498
6, avg labels: 0.01464 avg loss:0.01500, max loss: 0.07582, min loss : 0.00431
7, avg labels: 0.01464 avg loss:0.01509, max loss: 0.07544, min loss : 0.00428
8, avg labels: 0.01464 avg loss:0.01489, max loss: 0.07536, min loss : 0.00442
9, avg labels: 0.01464 avg loss:0.01501, max loss: 0.07591, min loss : 0.00478
10, avg labels: 0.01464 avg loss:0.01499, max loss: 0.07580, min loss : 0.00620
实际值:tensor([[-0.0121],
        [ 0.0155],
        [ 0.0056],
        [ 0.0096],
        [ 0.0420],
        [-0.0244],
        [ 0.0133],
        [-0.0038],
        [ 0.0317],
        [ 0.0247],
        [-0.019